# 1. DATA PREPARATION

## 1.1 Extract PDF from webpage "Ministerio de Sanidad"

In [ ]:
import os
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service

# 📁 Crear carpeta raíz
main_output_dir = os.path.abspath("examenes_mir_v_0")
os.makedirs(main_output_dir, exist_ok=True)

# 🌐 Configuración del navegador
options = webdriver.ChromeOptions()
prefs = {
    "download.prompt_for_download": False,
    "plugins.always_open_pdf_externally": True
}
options.add_experimental_option("prefs", prefs)

# 🚀 Iniciar navegador
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
wait = WebDriverWait(driver, 10)

# Abrir página inicial
driver.get("https://fse.mscbs.gob.es/fseweb/view/public/datosanteriores/cuadernosExamen/busquedaConvocatoria.xhtml")

# Obtener titulaciones disponibles
select_titulacion = Select(wait.until(EC.presence_of_element_located((By.ID, "mainForm:titulacionSelect"))))
titulaciones = [option.get_attribute("value") for option in select_titulacion.options if option.get_attribute("value")]

# Etiquetas de tipos de PDF
tipo_mapeo = {
    "Cuaderno de Texto": "cuaderno_texto",
    "Cuaderno de Imágenes": "cuaderno_imagenes"
}

# Iterar por titulaciones
for titulacion in titulaciones:
    print(f"📘 Procesando titulación: {titulacion}")

    # Volver a capturar el select por seguridad
    select_titulacion = Select(wait.until(EC.presence_of_element_located((By.ID, "mainForm:titulacionSelect"))))
    select_titulacion.select_by_value(titulacion)
    time.sleep(2)

    titulacion_dir = os.path.join(main_output_dir, titulacion)
    os.makedirs(titulacion_dir, exist_ok=True)

    # Obtener convocatorias
    select_convocatoria = Select(wait.until(EC.presence_of_element_located((By.ID, "mainForm:anyosSelect"))))
    convocatorias = [option.text.strip() for option in select_convocatoria.options if option.text.strip()]

    for convocatoria in convocatorias:
        print(f"  📅 Convocatoria: {convocatoria}")
        select_convocatoria = Select(wait.until(EC.presence_of_element_located((By.ID, "mainForm:anyosSelect"))))
        select_convocatoria.select_by_visible_text(convocatoria)
        time.sleep(1.5)

        try:
            # Intentar seleccionar solo la versión 0
            select_version = Select(wait.until(EC.presence_of_element_located((By.ID, "mainForm:versionSelect"))))
            version_0_found = any(opt.text.strip() == "0" for opt in select_version.options)

            if not version_0_found:
                print("    ⚠️ No hay versión 0, se omite.")
                continue

            select_version.select_by_visible_text("0")
            time.sleep(1)

            # Buscar tipo de PDF (texto o imagen)
            radios = driver.find_elements(By.XPATH, '//input[@type="radio" and contains(@id,"mainForm:j_idt87:")]')

            for radio in radios:
                radio_id = radio.get_attribute("id")
                label = driver.find_element(By.CSS_SELECTOR, f'label[for="{radio_id}"]')
                tipo_visible = label.text.strip()

                if tipo_visible in tipo_mapeo:
                    nombre_carpeta = tipo_mapeo[tipo_visible]
                    print(f"    📥 Descargando tipo: {nombre_carpeta}")

                    subcarpeta = os.path.join(titulacion_dir, nombre_carpeta)
                    os.makedirs(subcarpeta, exist_ok=True)

                    # Cambiar carpeta de descarga dinámica
                    driver.execute_cdp_cmd("Page.setDownloadBehavior", {
                        "behavior": "allow",
                        "downloadPath": subcarpeta
                    })

                    radio.click()
                    time.sleep(0.5)

                    # Clic en "Ver"
                    ver_btn = driver.find_element(By.ID, "mainForm:j_idt91")
                    ver_btn.click()
                    time.sleep(3)

        except Exception as e:
            print(f"    ❌ Error con {titulacion} - {convocatoria} v0: {e}")

driver.quit()
print("\n✅ Finalizado correctamente.")

## 1.2 Convert PDF to JSON

In [ ]:
import fitz  # PyMuPDF
import re
import json
import os

def extract_text_from_pdf(pdf_path):
    """Extrae texto completo de todas las páginas del PDF."""
    try:
        pdf_document = fitz.open(pdf_path)
        all_text = []

        for page_num in range(len(pdf_document)):
            page = pdf_document[page_num]
            text = page.get_text()
            all_text.append(text.strip())

        pdf_document.close()
        return "\n".join(all_text)

    except Exception as e:
        print(f"⚠️ Error al leer {pdf_path}: {e}")
        return ""

def extract_questions(text, nombre_pdf):
    """Extrae preguntas y opciones del texto usando regex."""
    patron = re.compile(
        r"\n?(\d+)\.\s*(.*?)\s*"
        r"1\.\s*(.*?)\s*"
        r"2\.\s*(.*?)\s*"
        r"3\.\s*(.*?)\s*"
        r"4\.\s*(.*?)(?=\n\d+\.|\Z)",
        re.DOTALL
    )

    preguntas = []
    for match in patron.finditer(text):
        numero = int(match.group(1))
        enunciado = match.group(2).strip().replace('\n', ' ')
        opciones = [match.group(i).strip().replace('\n', ' ') for i in range(3, 7)]

        preguntas.append({
            "numero": numero,
            "enunciado": enunciado,
            "opciones": opciones,
            "archivo_origen": nombre_pdf
        })

    return preguntas

def procesar_titulacion(titulacion_path, salida_dir):
    """Procesa todos los PDFs de texto en una titulación dada."""
    cuaderno_texto_path = os.path.join(titulacion_path, "cuaderno_texto")
    if not os.path.isdir(cuaderno_texto_path):
        print(f"⚠️ No se encontró carpeta cuaderno_texto en {titulacion_path}")
        return

    preguntas_total = []
    for filename in os.listdir(cuaderno_texto_path):
        if filename.lower().endswith(".pdf"):
            pdf_path = os.path.join(cuaderno_texto_path, filename)
            print(f"📄 Procesando: {pdf_path}")
            texto = extract_text_from_pdf(pdf_path)
            preguntas = extract_questions(texto, filename)
            preguntas_total.extend(preguntas)

    # Guardar JSON por titulación
    nombre_titulacion = os.path.basename(titulacion_path)
    output = {
        "titulacion": nombre_titulacion,
        "preguntas": preguntas_total
    }

    os.makedirs(salida_dir, exist_ok=True)
    json_path = os.path.join(salida_dir, f"{nombre_titulacion}.json")
    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(output, f, ensure_ascii=False, indent=2)

    print(f"✅ Guardado: {json_path} ({len(preguntas_total)} preguntas)")

def main():
    base_dir = "examenes_mir_v_0"       # Carpeta raíz donde están las titulaciones
    salida_dir = "results/1_data_preparation/1_json_por_titulacion"  # Carpeta de salida

    for titulacion in os.listdir(base_dir):
        titulacion_path = os.path.join(base_dir, titulacion)
        if os.path.isdir(titulacion_path):
            print(f"\n🧪 Procesando titulación: {titulacion}")
            procesar_titulacion(titulacion_path, salida_dir)

if __name__ == "__main__":
    main()

## 1.3 Extract correct answers

In [ ]:
import os
import json
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

# Carpeta de salida
os.makedirs("results/1_data_preparation/2_respuestas_json", exist_ok=True)

# Configuración del navegador (headless para evitar abrir ventanas)
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")

# URL base
URL_ANTERIORES = "https://fse.mscbs.gob.es/fseweb/view/public/datosanteriores/cuadernosExamen/busquedaConvocatoria.xhtml"

def extraer_respuestas(titulacion, convocatoria, driver, wait):
    try:
        if convocatoria != "2024":
            driver.get(URL_ANTERIORES)
            print("    🧭 Página cargada.")

            Select(wait.until(EC.presence_of_element_located((By.ID, "mainForm:titulacionSelect")))).select_by_visible_text(titulacion)
            time.sleep(1)

            Select(wait.until(EC.presence_of_element_located((By.ID, "mainForm:anyosSelect")))).select_by_visible_text(convocatoria)
            time.sleep(1)

            select_version = Select(wait.until(EC.presence_of_element_located((By.ID, "mainForm:versionSelect"))))
            versiones = [opt.text.strip() for opt in select_version.options if opt.get_attribute("value")]
            if "0" not in versiones:
                print("    ⚠️ No hay versión 0, se omite.")
                return
            select_version.select_by_visible_text("0")
            time.sleep(1)

            radios = driver.find_elements(By.XPATH, '//input[@type="radio" and contains(@id,"mainForm:j_idt87:")]')
            for radio in radios:
                label = driver.find_element(By.CSS_SELECTOR, f'label[for="{radio.get_attribute("id")}"]')
                if "Hoja de Respuestas" in label.text:
                    radio.click()
                    break
            else:
                print("    ⚠️ No se encontró radio de Hoja de Respuestas.")
                return

            wait.until(EC.element_to_be_clickable((By.ID, "mainForm:j_idt91"))).click()
            time.sleep(2)

        else:

            print("    ↪ Entrando en modo especial para 2024...")

            driver.get(URL_ANTERIORES)
            print("    🧭 Página cargada.")

            Select(wait.until(EC.presence_of_element_located((By.ID, "mainForm:titulacionSelect")))).select_by_visible_text(titulacion)
            time.sleep(1)

            Select(wait.until(EC.presence_of_element_located((By.ID, "mainForm:anyosSelect")))).select_by_visible_text(convocatoria)
            time.sleep(1)

            select_version = Select(wait.until(EC.presence_of_element_located((By.ID, "mainForm:versionSelect"))))
            versiones = [opt.text.strip() for opt in select_version.options if opt.get_attribute("value")]
            if "0" not in versiones:
                print("    ⚠️ No hay versión 0, se omite.")
                return
            select_version.select_by_visible_text("0")
            time.sleep(1)

            radios = driver.find_elements(By.XPATH, '//input[@type="radio" and contains(@id,"mainForm:j_idt87:")]')
            for radio in radios:
                label = driver.find_element(By.CSS_SELECTOR, f'label[for="{radio.get_attribute("id")}"]')
                if "Hoja de Respuestas" in label.text:
                    radio.click()
                    break
            else:
                print("    ⚠️ No se encontró radio de Hoja de Respuestas.")
                return

            wait.until(EC.element_to_be_clickable((By.ID, "mainForm:j_idt91"))).click()
            time.sleep(2)

            try:
                acceso = wait.until(EC.element_to_be_clickable((By.ID, "mainForm:j_idt93")))
                acceso.click()
                print("    🔓 Click en 'Para consultar las respuestas correctas de la convocatoria actual, pulse aqui.'")
            except Exception as e:
                print("    ❌ Fallo en 'Para consultar las respuestas correctas de la convocatoria actual, pulse aqui.'", e)
                return

            try:
                acceso = wait.until(EC.element_to_be_clickable((By.ID, "mainForm:accAbierto")))
                acceso.click()
                print("    🔓 Click en 'Acceso sin identificación'")
            except Exception as e:
                print("    ❌ Fallo en acceso sin identificación:", e)
                return


            time.sleep(2)

            try:
                sel_titulacion = Select(wait.until(EC.presence_of_element_located((By.ID, "mainForm:titulaciones"))))
                sel_titulacion.select_by_visible_text(titulacion)
                print(f"    ✅ Titulación seleccionada: {titulacion}")
            except Exception as e:
                print("    ❌ Fallo al seleccionar titulación:", e)
                return

            time.sleep(1)

            try:
                # Esperar y seleccionar el desplegable por name
                sel_version = Select(wait.until(EC.presence_of_element_located(
                    (By.XPATH, '//*[@id="mainForm:versiones"]/select')
                )))

                versiones = [opt.text.strip() for opt in sel_version.options if opt.get_attribute("value")]
                if "0" not in versiones:
                    sel_version.select_by_visible_text("1")
                    time.sleep(1)
                else:
                    sel_version.select_by_visible_text("0")
                    time.sleep(1)

            except Exception as e:
                print("    ❌ Fallo al seleccionar versión:", e)
                return

            time.sleep(1)

            try:
                buscar = wait.until(EC.element_to_be_clickable(
                    (By.ID, 'mainForm:j_idt100')
                ))

                time.sleep(1)

                buscar.click()
                print("    🔍 Click en botón Buscar.")
            except Exception as e:
                print("    ❌ Fallo al hacer click en buscar:", e)
                return

            time.sleep(2)


        # Extraer respuestas
        soup = BeautifulSoup(driver.page_source, "html.parser")
        tabla = soup.find("table")
        if not tabla:
            print("    ⚠️ No se encontró tabla de respuestas.")
            return

        respuestas = {}
        celdas = tabla.find_all("td")
        for i in range(0, len(celdas), 2):
            try:
                num = int(celdas[i].text.strip())
                val = int(celdas[i + 1].text.strip())
                respuestas[num] = val
            except:
                continue

        # Guardar archivo
        nombre_archivo = f"{titulacion}_{convocatoria}.json".replace(" ", "_")
        ruta_salida = os.path.join("2_respuestas_json", nombre_archivo)

        with open(ruta_salida, "w", encoding="utf-8") as f:
            json.dump(respuestas, f, indent=2, ensure_ascii=False)

        print(f"    ✅ Respuestas guardadas: {nombre_archivo}")

    except Exception as e:
        print(f"    ❌ Error procesando {titulacion} {convocatoria}: {e}")

# Inicialización de navegador
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
wait = WebDriverWait(driver, 10)

try:
    driver.get(URL_ANTERIORES)
    select_titulacion = Select(wait.until(EC.presence_of_element_located((By.ID, "mainForm:titulacionSelect"))))
    titulaciones = [opt.text.strip() for opt in select_titulacion.options if opt.get_attribute("value")]

    for titulacion in titulaciones:
        print(f"\n🧪 Titulación: {titulacion}")

        driver.get(URL_ANTERIORES)
        Select(wait.until(EC.presence_of_element_located((By.ID, "mainForm:titulacionSelect")))).select_by_visible_text(titulacion)
        time.sleep(1)

        select_convocatoria = Select(wait.until(EC.presence_of_element_located((By.ID, "mainForm:anyosSelect"))))
        convocatorias = [opt.text.strip() for opt in select_convocatoria.options if opt.get_attribute("value")]

        for convocatoria in convocatorias:
            print(f"  📅 Convocatoria: {convocatoria}")
            extraer_respuestas(titulacion, convocatoria, driver, wait)

finally:
    driver.quit()
    print("\n🏁 Finalizado.")

## 1.4 Add correct answer to JSON

In [ ]:
import os
import json
import re

# Carpetas
carpeta_preguntas = "results/1_data_preparation/1_json_por_titulacion"     # Donde están los JSON con las preguntas
carpeta_respuestas = "results/1_data_preparation/2_respuestas_json"        # Donde están los JSON con las respuestas (solo versión 0)
carpeta_salida = "results/1_data_preparation/3_json_con_respuesta"         # Carpeta donde se guardarán los JSON con respuestas añadidas
os.makedirs(carpeta_salida, exist_ok=True)      # Crear carpeta de salida si no existe

# === CARGAR RESPUESTAS EN MEMORIA ===
respuestas_por_archivo = {}  # Diccionario para guardar respuestas por archivo

for archivo in os.listdir(carpeta_respuestas):
    if archivo.endswith(".json"):  # Solo JSON
        ruta = os.path.join(carpeta_respuestas, archivo)
        with open(ruta, "r", encoding="utf-8") as f:
            respuestas_por_archivo[archivo] = {
                int(k): v for k, v in json.load(f).items()
            }

# === PROCESAR CADA ARCHIVO DE PREGUNTAS ===
for archivo in os.listdir(carpeta_preguntas):
    if not archivo.endswith(".json"):
        continue

    ruta = os.path.join(carpeta_preguntas, archivo)
    with open(ruta, "r", encoding="utf-8") as f:
        data = json.load(f)

    titulacion = data.get("titulacion", "").strip().upper()
    preguntas = data.get("preguntas", [])

    for pregunta in preguntas:
        archivo_origen = pregunta.get("archivo_origen", "")
        numero = pregunta.get("numero")

        # Buscar año desde el nombre del PDF: "Cuaderno_2020_MEDICINA_0_C.pdf"
        match = re.search(r"Cuaderno_(\d{4})_" + re.escape(titulacion) + r"_\d+_C", archivo_origen)
        if not match:
            pregunta["respuesta_correcta"] = None
            continue

        año = match.group(1)

        # El nuevo nombre de archivo de respuestas solo tiene año, sin _v0
        archivo_respuestas = f"{titulacion}_{año}.json"

        respuestas = respuestas_por_archivo.get(archivo_respuestas)
        if respuestas:
            pregunta["respuesta_correcta"] = respuestas.get(numero, None)
        else:
            pregunta["respuesta_correcta"] = None

        # 👉 Añadir etiquetas de titulación y año
        pregunta["titulacion"] = titulacion
        pregunta["convocatoria"] = año

    # Guardar salida
    salida = os.path.join(carpeta_salida, archivo)
    with open(salida, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

    print(f"✅ Guardado en {salida}")


## 1.5 Remove instructions

In [ ]:
import os
import json
import re

CARPETA_ENTRADA = "results/1_data_preparation/3_json_con_respuesta"
CARPETA_SALIDA = "results/1_data_preparation/4_json_corregido"

os.makedirs(CARPETA_SALIDA, exist_ok=True)

def separar_enunciado_y_opcion(texto):
    # Elimina marcador inicial como "1." o "A."
    match = re.search(r"^(.*?)(?:\s*[1A]\.\s*)(.+)$", texto.strip())
    if match:
        enunciado = match.group(1).strip()
        primera_opcion = match.group(2).strip()
        return enunciado, primera_opcion
    else:
        return "", texto.strip()

for nombre_archivo in os.listdir(CARPETA_ENTRADA):
    if not nombre_archivo.endswith(".json"):
        continue

    ruta_entrada = os.path.join(CARPETA_ENTRADA, nombre_archivo)
    with open(ruta_entrada, "r", encoding="utf-8") as f:
        data = json.load(f)

    for pregunta in data.get("preguntas", []):
        if pregunta.get("numero") == 1:
            opciones = pregunta.get("opciones", [])
            if opciones:
                nuevo_enunciado, primera_opcion = separar_enunciado_y_opcion(opciones[0])
                if nuevo_enunciado:
                    pregunta["enunciado"] = nuevo_enunciado
                    pregunta["opciones"][0] = primera_opcion
                else:
                    pregunta["enunciado"] = ""

    ruta_salida = os.path.join(CARPETA_SALIDA, nombre_archivo)
    with open(ruta_salida, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

    print(f"✅ Corregido: {nombre_archivo}")

## 1.6 Add type text or image

In [ ]:
import os
import json

input_dir = "results/1_data_preparation/4_json_corregido"
output_dir = "results/1_data_preparation/5_json_type"

os.makedirs(output_dir, exist_ok=True)

titulaciones_con_imagenes = {"MEDICINA", "ENFERMERÍA"}

# Contadores globales
total_global = 0
texto_global = 0
imagen_global = 0

for filename in os.listdir(input_dir):
    if filename.endswith(".json"):
        with open(os.path.join(input_dir, filename), "r", encoding="utf-8") as f:
            data = json.load(f)

        titulacion = data.get("titulacion", "").upper()

        total = 0
        texto = 0
        imagen = 0

        for pregunta in data.get("preguntas", []):
            total += 1
            enunciado = pregunta.get("enunciado", "").lower()

            if titulacion in titulaciones_con_imagenes and "pregunta asociada a la imagen" in enunciado:
                pregunta["tipo"] = "imagen"
                imagen += 1
            else:
                pregunta["tipo"] = "texto"
                texto += 1

        # Actualizar contadores globales
        total_global += total
        texto_global += texto
        imagen_global += imagen

        # Guardar JSON actualizado
        output_path = os.path.join(output_dir, filename)
        with open(output_path, "w", encoding="utf-8") as f:
            json.dump(data, f, ensure_ascii=False, indent=2)

        print(f"✅ {filename} procesado:")
        print(f"   📊 Total preguntas: {total}")
        print(f"   🖼️ Imagen: {imagen} | 📝 Texto: {texto}")

# Mostrar resumen global
print("\n📦 RESUMEN GLOBAL:")
print(f"🔢 Total preguntas procesadas: {total_global}")
print(f"🖼️ Total tipo imagen: {imagen_global}")
print(f"📝 Total tipo texto: {texto_global}")

if total_global > 0:
    p_imagen = round((imagen_global / total_global) * 100, 2)
    p_texto = round((texto_global / total_global) * 100, 2)
    print(f"\n📊 Porcentajes globales:")
    print(f"   🖼️ Imagen: {p_imagen}%")
    print(f"   📝 Texto: {p_texto}%")
else:
    print("⚠️ No se procesaron preguntas.")

## 1.7 Add titulation and year

In [ ]:
import os
import json
import re

carpeta_entrada = "results/1_data_preparation/5_json_type"
carpeta_salida = "results/1_data_preparation/6_json_final"
os.makedirs(carpeta_salida, exist_ok=True)

for nombre_archivo in os.listdir(carpeta_entrada):
    if not nombre_archivo.endswith(".json"):
        continue

    ruta = os.path.join(carpeta_entrada, nombre_archivo)

    with open(ruta, "r", encoding="utf-8") as f:
        data = json.load(f)

    titulacion = data.get("titulacion", "").strip().upper()
    preguntas = data.get("preguntas", [])

    año = "DESCONOCIDO"
    for pregunta in preguntas:
        archivo_origen = pregunta.get("archivo_origen", "")
        match = re.search(r"Cuaderno_(\d{4})_" + re.escape(titulacion) + r"_\d+_C", archivo_origen)
        if match:
            año = match.group(1)
            break  # Ya encontrado, salimos

    # Añadir etiquetas al JSON
    data["etiqueta_titulacion"] = titulacion
    data["etiqueta_convocatoria"] = año

    # Guardar en carpeta nueva
    ruta_salida = os.path.join(carpeta_salida, nombre_archivo)
    with open(ruta_salida, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

    print(f"✅ {nombre_archivo} etiquetado como {titulacion} - {año}")

print("\n📦 Todos los archivos procesados y guardados en '6_json_final'.")


## 1.8 Find images FARMACIA

In [ ]:
import fitz  # PyMuPDF
import re
import os
from PIL import Image
import pytesseract

# Parámetros
pdf_path = "Cuaderno_2023_FARMACIA_0_C.pdf"  # Ruta local al PDF
output_dir = "imagenes_farmacia_detectadas"
os.makedirs(output_dir, exist_ok=True)

# Abrir PDF
doc = fitz.open(pdf_path)

# Resultado
preguntas_con_imagen = []

# Recorrer páginas
for i, page in enumerate(doc):
    images = page.get_images(full=True)
    if not images:
        continue  # No hay imágenes

    # Extraer imagen de página
    pix = page.get_pixmap(dpi=300)
    image_path = os.path.join(output_dir, f"pagina_{i+1}.png")
    pix.save(image_path)

    # OCR para detectar número de pregunta (sin lang="spa")
    text = pytesseract.image_to_string(Image.open(image_path))
    match = re.search(r"\b(\d{1,3})\.", text)
    numero_pregunta = int(match.group(1)) if match else None

    preguntas_con_imagen.append({
        "pagina": i + 1,
        "pregunta_detectada": numero_pregunta,
        "imagen_extraida": image_path
    })

# Mostrar resumen
for info in preguntas_con_imagen:
    print(f"📄 Página {info['pagina']}: Pregunta {info['pregunta_detectada']} -> Imagen guardada en {info['imagen_extraida']}")
